In [1]:
import pandas as pd
import numpy as np
from numpy import nan
import json
import re


In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
filename = '/Users/rupalgandhre/SpringBoard/DataScience_Capstone2/data/raw/effy_all_jewelry_cat.csv'
raw_data = pd.read_csv(filename )


In [4]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2471 entries, 0 to 2470
Columns: 146 entries, Description to 25" Gemstone
dtypes: float64(5), object(141)
memory usage: 2.8+ MB


In [5]:
raw_data.columns

Index(['Description', 'Discount_Price', 'Price', 'Metal', 'Metal Color',
       'Stones', 'Length', 'Hoop Size', 'Height 3/8" x Length', 'Diameter',
       ...
       '18" Chain Included. Heart Length', '18" Chain Included. Size of Heart',
       '16" - 18" Adjustable Chain. Butterfly Size', ' Necklace Length',
       '16"-18" Adjustable Chain Included. Pendant Legnth',
       '17" Chain Included. Pendant Length',
       '17" Attached Chain. Pendant Length',
       '4 Strand 18" Necklace. Pearl Sizes', 'Pendant Length', '25" Gemstone'],
      dtype='object', length=146)

In [6]:
raw_data.head(1)

,Description,Discount_Price,Price,Metal,Metal Color,Stones,Length,Hoop Size,"Height 3/8"" x Length",Diameter,...,"18"" Chain Included. Heart Length","18"" Chain Included. Size of Heart","16"" - 18"" Adjustable Chain. Butterfly Size",Necklace Length,"16""-18"" Adjustable Chain Included. Pendant Legnth","17"" Chain Included. Pendant Length","17"" Attached Chain. Pendant Length","4 Strand 18"" Necklace. Pearl Sizes",Pendant Length,"25"" Gemstone"
0,"Effy Brasilica 14K Yellow Gold Emerald and Diamond Earrings, 4.62 TCW",5596.5,7995.0,14K Gold,Yellow,"Multi Shape Emerald 4.01, Round Diamond 0.61","1.5""",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Explore The Data

In [7]:
#Count the number of missing values in each column and sort them.
missing = pd.concat([raw_data.isnull().sum(), 100 * raw_data.isnull().mean()], axis=1)
missing.columns=['count', '%']
missing.sort_values(by='count', ascending=True).head(20)


,count,%
Description,0,0.000000
Sex,0,0.000000
Jewelry_Type,0,0.000000
Metal,2,0.080939
Discount_Price,13,0.526103
Stones,17,0.687981
Price,40,1.618778
Metal Color,221,8.943747
Top Width,2094,84.743019
Length,2109,85.350061


In [8]:
raw_data.select_dtypes('object')

,Description,Metal,Metal Color,Stones,Length,Hoop Size,"Height 3/8"" x Length",Diameter,Earring,Drop Length,...,"Adjustable 18"" Pull Chain. Evil Eye Length","18"" Chain Included. Heart Length","18"" Chain Included. Size of Heart","16"" - 18"" Adjustable Chain. Butterfly Size",Necklace Length,"16""-18"" Adjustable Chain Included. Pendant Legnth","17"" Chain Included. Pendant Length","17"" Attached Chain. Pendant Length","4 Strand 18"" Necklace. Pearl Sizes","25"" Gemstone"
0,"Effy Brasilica 14K Yellow Gold Emerald and Diamond Earrings, 4.62 TCW",14K Gold,Yellow,"Multi Shape Emerald 4.01, Round Diamond 0.61","1.5""",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Effy Pave Classica 14K White Gold Diamond Hoop Earrings, 0.98 TCW",14K Gold,White,"Round Diamond 0.98,",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Effy Brasilica 14K White Gold Emerald and Diamond Drop Earrings, 4.04 TCW",14K Gold,Yellow,"Round Diamond 1.38, Pear Emerald 2.66,",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Effy Canare 14K Yellow Gold Yellow Diamond Hoop Earrings, 1.15 TCW",14K Gold,Yellow,Round Diamond 1.15,NaN,"3/4""",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Effy Tanzanite Royale 14K Gold Tanzanite and Diamond Earrings, 2.90 TCW",14K Gold,White,"Multi Shape Tanzanite 2.6, Round Diamond 0.3","3/4""",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2466,"Effy Mosaic 14K Yellow Gold Multi Gemstone and Diamond Pendant, 1.18 TCW",14K Gold,Yellow,"Square Multi Color 1.1, Round Diamond 0.08",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2467,"Effy Bordeaux 14K Yellow Gold Garnet and Diamond Pendant, 8.14 TCW",14K Gold,Yellow,"Emerald Cut Garnet 7.85, Multi Shape Diamond 0.29",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2468,"Effy Watercolors 14K Yellow Gold Multi Sapphire & Diamond Pendant, 1.85 TCW",14K Gold,Yellow,"Oval Multi Sapphire 1.59, Round Diamond 0.26",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2469,"Effy Trio 14K Tri Color Gold Diamond Necklace, 0.40 TCW",14K Gold,Three Tone,Multi Shape Diamond 0.4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#Check if there are duplicate products
raw_data['Description'].value_counts().head(20)

Effy Blush 14K Rose Gold Morganite and Diamond Ring, 2.64 TCW                 3
Effy Pave Classica 14K White Gold Diamond Ring, 0.32 TCW                      3
Effy 925 Sterling Silver & 18K Yellow Gold Cultured Fresh Water Pearl Ring    2
Effy Pave Rose 14K Rose Gold Diamond Ring, 0.37 TCW                           2
Effy Pave Classica 14K White Gold Diamond Criss Cross Ring, 0.40 TCW          2
Effy Ruby Royale 14K Yellow Gold Ruby and Diamond Ring, 1.36 TCW              2
Effy 14K Yellow Gold Cultured Fresh Water Pearl Necklace                      2
Effy 14K Yellow Gold Cultured Fresh Water Pearl Earrings                      2
Effy 14K Yellow Gold Cultured Fresh Water Pearl Bracelet                      2
Effy Watercolors 14K Yellow Gold Multi Sapphire and Diamond Earrings          2
Effy Trio 14K Tri Color Gold Diamond Ring, 0.98 TCW                           2
Effy Seaside 14K White Gold Blue Sapphire & Diamond Starfish Pendant          2
Effy Ruby Royale 14K Yellow Gold Ruby St

In [10]:
raw_data.loc[raw_data['Description'] =='Effy Pave Classica 14K White Gold Diamond Ring, 0.32 TCW'][['Description','Discount_Price', 'Price','Metal','Metal Color','Stones']]


,Description,Discount_Price,Price,Metal,Metal Color,Stones
598,"Effy Pave Classica 14K White Gold Diamond Ring, 0.32 TCW",1326.5,1895.0,14K Gold,White,Multi Shape Diamond 0.32
915,"Effy Pave Classica 14K White Gold Diamond Ring, 0.32 TCW",1125.0,2250.0,14K Gold,White,"Round Diamond 0.32,"
921,"Effy Pave Classica 14K White Gold Diamond Ring, 0.32 TCW",1547.5,3095.0,14K Gold,White,"Round Diamond 0.32,"


In [11]:
raw_data.loc[raw_data['Description'] == 'Effy Blush 14K Rose Gold Morganite and Diamond Ring, 2.64 TCW'][['Description','Discount_Price', 'Price','Metal','Metal Color','Stones']]



,Description,Discount_Price,Price,Metal,Metal Color,Stones
617,"Effy Blush 14K Rose Gold Morganite and Diamond Ring, 2.64 TCW",1676.5,2395.0,14K Gold,Rose,"Emerald Cut Morganite 2.55, Round Diamond 0.09,"
704,"Effy Blush 14K Rose Gold Morganite and Diamond Ring, 2.64 TCW",1956.5,2795.0,14K Gold,Rose,"Oval Morganite 2.35, Round Diamond 0.29"
1495,"Effy Blush 14K Rose Gold Morganite and Diamond Ring, 2.64 TCW",1347.5,2695.0,14K Gold,Rose,"Oval Morganite 2.33, Round Cognac Diamond 0.17, Round Diamond 0.14,"


## Observation from Price being null: 
If Discount_Price and Price both are NaN --> Product is sold out --> These records are dropped 
If only Price is Nan --> Discount_Price is final price and product is not on sale --> The 'Price' for this product is updated with 'Discount_Price'

In [12]:
#Check for records with null price
raw_data.loc[raw_data['Price'].isnull() | raw_data['Discount_Price'].isnull()]

,Description,Discount_Price,Price,Metal,Metal Color,Stones,Length,Hoop Size,"Height 3/8"" x Length",Diameter,...,"18"" Chain Included. Heart Length","18"" Chain Included. Size of Heart","16"" - 18"" Adjustable Chain. Butterfly Size",Necklace Length,"16""-18"" Adjustable Chain Included. Pendant Legnth","17"" Chain Included. Pendant Length","17"" Attached Chain. Pendant Length","4 Strand 18"" Necklace. Pearl Sizes",Pendant Length,"25"" Gemstone"
13,"Effy Pave Classica 14K White Gold Diamond 2"" Hoop Earrings, 2.65 TCW",NaN,NaN,14K Gold,White,"Round Diamond 2.65,",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,"Effy Ruby Royale 14K Rose Gold Ruby and Diamond Hoop Earrings, 0.55 TCW",NaN,NaN,14K Gold,Rose,"Round Ruby 0.31, Round Diamond 0.24","5/8""",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
159,"Effy 925 Sterling Silver Pink Sapphire Splash Small Hoop Earrings, 1.80 TCW",295.0,NaN,Sterling Silver,NaN,Round Pink Sapphire 1.8,NaN,NaN,NaN,"5/8""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
262,"Effy 925 Sterling Silver Blue Sapphire Splash 1/2"" Hoop Earrings, 1.77 TCW",295.0,NaN,Sterling Silver,NaN,Round Blue Sapphire 1.77,NaN,NaN,NaN,"5/8""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338,"Effy 14K White Gold Diamond Stud Earring, 2.00 TCW",6995.0,NaN,14K Gold,White,Round Diamond 2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
433,"Effy D'Oro 14K Yellow Gold Diamond Flower Ring, 1.61 TCW",NaN,NaN,14K Gold,Yellow,Round Diamond 1.61,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
471,"Effy 14K White Gold Diamond Band, 0.58 TCW",1395.0,NaN,14K Gold,White,"Round Diamond 0.58,",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
481,"Effy Bridal 14K White Gold Diamond Solitaire Ring, 0.20 TCW",850.0,NaN,14K Gold,White,Round Diamond 0.2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506,"Effy Pave Classica 14K White Gold Diamond Ring, 2.18 TCW",NaN,NaN,14K Gold,White,Round Diamond 2.18,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
521,"Effy Pave Classica 14K White Gold Diamond Ring, 1.42 TCW",6250.0,NaN,14K Gold,White,"Round Diamond 0.77, Baguette Diamond 0.65,",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#If both the prices are NaN then the product is sold-out. Remove these records
missing_price = raw_data[['Price', 'Discount_Price']].isnull().sum(axis=1)
missing_price.value_counts()/len(missing_price) * 100

0    98.381222
1     1.092675
2     0.526103
dtype: float64

In [14]:
raw_data = raw_data[missing_price != 2]

In [15]:
raw_data.loc[raw_data['Price'].isnull() & raw_data['Discount_Price'].isnull()]

,Description,Discount_Price,Price,Metal,Metal Color,Stones,Length,Hoop Size,"Height 3/8"" x Length",Diameter,...,"18"" Chain Included. Heart Length","18"" Chain Included. Size of Heart","16"" - 18"" Adjustable Chain. Butterfly Size",Necklace Length,"16""-18"" Adjustable Chain Included. Pendant Legnth","17"" Chain Included. Pendant Length","17"" Attached Chain. Pendant Length","4 Strand 18"" Necklace. Pearl Sizes",Pendant Length,"25"" Gemstone"


In [16]:
#If Price is NaN then the product is not on sale. Update these records with 'Discount_Price'
raw_data.loc[raw_data['Price'].isnull(), 'Price'] = raw_data['Discount_Price']

In [17]:
raw_data.loc[raw_data['Description']=='Effy 14K White Gold Diamond Stud Earring, 2.00 TCW']

,Description,Discount_Price,Price,Metal,Metal Color,Stones,Length,Hoop Size,"Height 3/8"" x Length",Diameter,...,"18"" Chain Included. Heart Length","18"" Chain Included. Size of Heart","16"" - 18"" Adjustable Chain. Butterfly Size",Necklace Length,"16""-18"" Adjustable Chain Included. Pendant Legnth","17"" Chain Included. Pendant Length","17"" Attached Chain. Pendant Length","4 Strand 18"" Necklace. Pearl Sizes",Pendant Length,"25"" Gemstone"
338,"Effy 14K White Gold Diamond Stud Earring, 2.00 TCW",6995.0,6995.0,14K Gold,White,Round Diamond 2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Observation on Metal Color being null
All Sterling Silver products have Metal Color as Nan --> The Metal Color for this products is replaced with their respective Metal

If 'Metal' and 'Metal Color' both are Nan --> This products are made of Pearls

In [18]:
raw_data.loc[raw_data['Metal Color'].isnull() & raw_data['Metal'].isnull()]

,Description,Discount_Price,Price,Metal,Metal Color,Stones,Length,Hoop Size,"Height 3/8"" x Length",Diameter,...,"18"" Chain Included. Heart Length","18"" Chain Included. Size of Heart","16"" - 18"" Adjustable Chain. Butterfly Size",Necklace Length,"16""-18"" Adjustable Chain Included. Pendant Legnth","17"" Chain Included. Pendant Length","17"" Attached Chain. Pendant Length","4 Strand 18"" Necklace. Pearl Sizes",Pendant Length,"25"" Gemstone"
2414,"Effy Cultured Fresh Water Pearl 40"" Necklace",245.0,350.0,NaN,NaN,Round Pearl,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2440,Effy Multi Color Cultured Fresh Water Pearl Necklace,206.5,295.0,NaN,NaN,Round Pearl,"36""",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
raw_data.loc[raw_data['Metal Color'].isnull() & raw_data['Metal'].isnull(), ['Metal', 'Metal Color']] =raw_data['Stones']

In [20]:
raw_data.loc[raw_data['Description'] == 'Effy Cultured Fresh Water Pearl 40" Necklace']

,Description,Discount_Price,Price,Metal,Metal Color,Stones,Length,Hoop Size,"Height 3/8"" x Length",Diameter,...,"18"" Chain Included. Heart Length","18"" Chain Included. Size of Heart","16"" - 18"" Adjustable Chain. Butterfly Size",Necklace Length,"16""-18"" Adjustable Chain Included. Pendant Legnth","17"" Chain Included. Pendant Length","17"" Attached Chain. Pendant Length","4 Strand 18"" Necklace. Pearl Sizes",Pendant Length,"25"" Gemstone"
2414,"Effy Cultured Fresh Water Pearl 40"" Necklace",245.0,350.0,Round Pearl,Round Pearl,Round Pearl,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
 raw_data.loc[raw_data['Metal Color'].isnull(), 'Metal'].unique()

array(['Sterling Silver', 'Sterling Silver & 18K Gold',
       'Stainless Steel & 18K Gold', '14K Gold'], dtype=object)

In [22]:
raw_data.loc[raw_data['Metal Color'].isnull() | raw_data['Metal'].isnull()]

,Description,Discount_Price,Price,Metal,Metal Color,Stones,Length,Hoop Size,"Height 3/8"" x Length",Diameter,...,"18"" Chain Included. Heart Length","18"" Chain Included. Size of Heart","16"" - 18"" Adjustable Chain. Butterfly Size",Necklace Length,"16""-18"" Adjustable Chain Included. Pendant Legnth","17"" Chain Included. Pendant Length","17"" Attached Chain. Pendant Length","4 Strand 18"" Necklace. Pearl Sizes",Pendant Length,"25"" Gemstone"
15,"Effy Splash Sterling Silver Pink Sapphire Circle Earrings, 3.60 TCW",665.0,950.0,Sterling Silver,NaN,Round Pink Sapphire 3.6,NaN,NaN,NaN,"1/2""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,"Effy Splash Sterling Silver Multi Color Sapphire Square Earrings, 3.75 TCW",836.5,1195.0,Sterling Silver,NaN,Round Multi Color 3.75,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,"Effy Tanzanite Royale Sterling Silver Tanzanite Stud Earrings, 1.52 TCW",367.5,525.0,Sterling Silver,NaN,Trillion Tanzanite 1.52,"1/4""",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,"Effy Splash Silver Blue Sapphire 1"" Hoop Earrings, 3.97 TCW",696.5,995.0,Sterling Silver,NaN,"Round Blue Sapphire 3.97,",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,Effy 925 Sterling Silver & 18K Yellow Gold Cultured Fresh Water Pearl Stud Earrings,332.5,475.0,Sterling Silver & 18K Gold,NaN,Round Pearl,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2433,"Effy Sterling Silver and 18K Yellow Gold Round Onyx Pendant, 36.50 TCW",547.5,1095.0,Sterling Silver & 18K Gold,NaN,Round Onyx 36.5,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,"1.75""",NaN,NaN,NaN,NaN
2436,"Effy 925 Sterling Silver Blue Sapphire Double Triangle Necklace, 1.30 TCW",287.5,575.0,Sterling Silver,NaN,Round Blue Sapphire 1.3,"18""",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2441,"Effy 925 Sterling Silver Sapphire and Diamond Necklace, 0.66 TCW",357.5,715.0,Sterling Silver,NaN,"Round Blue Sapphire 0.57, Round Diamond 0.09","19""",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2443,"Effy Seaside Sterling Silver Multi Gemstone Crab Pendant, 1.75 TCW",402.5,575.0,Sterling Silver,NaN,"Oval Amethyst 1.1, Round Pink Sapphire 0.65",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
 raw_data.loc[raw_data['Metal Color'].isnull(), 'Metal Color'] = raw_data['Metal']

In [24]:
raw_data.loc[raw_data['Metal Color'].isnull() | raw_data['Metal'].isnull()]

,Description,Discount_Price,Price,Metal,Metal Color,Stones,Length,Hoop Size,"Height 3/8"" x Length",Diameter,...,"18"" Chain Included. Heart Length","18"" Chain Included. Size of Heart","16"" - 18"" Adjustable Chain. Butterfly Size",Necklace Length,"16""-18"" Adjustable Chain Included. Pendant Legnth","17"" Chain Included. Pendant Length","17"" Attached Chain. Pendant Length","4 Strand 18"" Necklace. Pearl Sizes",Pendant Length,"25"" Gemstone"


In [25]:
raw_data.loc[raw_data['Description'] == 'Effy Splash Sterling Silver Pink Sapphire Circle Earrings, 3.60 TCW' ]

,Description,Discount_Price,Price,Metal,Metal Color,Stones,Length,Hoop Size,"Height 3/8"" x Length",Diameter,...,"18"" Chain Included. Heart Length","18"" Chain Included. Size of Heart","16"" - 18"" Adjustable Chain. Butterfly Size",Necklace Length,"16""-18"" Adjustable Chain Included. Pendant Legnth","17"" Chain Included. Pendant Length","17"" Attached Chain. Pendant Length","4 Strand 18"" Necklace. Pearl Sizes",Pendant Length,"25"" Gemstone"
15,"Effy Splash Sterling Silver Pink Sapphire Circle Earrings, 3.60 TCW",665.0,950.0,Sterling Silver,Sterling Silver,Round Pink Sapphire 3.6,NaN,NaN,NaN,"1/2""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Observation on Stones
The 'Stones' value is Nan, if there are no stones in the products -- The 'Stones' column is updated with 'No Stones'

In [26]:
raw_data.loc[raw_data['Stones'].isnull()]

,Description,Discount_Price,Price,Metal,Metal Color,Stones,Length,Hoop Size,"Height 3/8"" x Length",Diameter,...,"18"" Chain Included. Heart Length","18"" Chain Included. Size of Heart","16"" - 18"" Adjustable Chain. Butterfly Size",Necklace Length,"16""-18"" Adjustable Chain Included. Pendant Legnth","17"" Chain Included. Pendant Length","17"" Attached Chain. Pendant Length","4 Strand 18"" Necklace. Pearl Sizes",Pendant Length,"25"" Gemstone"
135,Effy 925 Classic Sterling Silver and 18K Yellow Gold Earrings,486.5,695.0,Sterling Silver & 18K Gold,Yellow,NaN,"3/4""",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,Effy 925 Sterling Silver & 18K Gold Cultured Pearl Earrings,332.5,475.0,Sterling Silver,Sterling Silver,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
263,Effy 925 Sterling Silver and 18K Yellow Gold Earrings,346.5,495.0,Sterling Silver & 18K Gold,Yellow,NaN,"3/4""",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
373,Effy 925 Sterling Silver and 18K Yellow Gold Accented Hoop Earrings,385.0,550.0,Sterling Silver & 18K Gold,Yellow,NaN,NaN,NaN,NaN,"1.25""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
905,Effy 925 Classic Sterling Silver and 18K Yellow Gold Ring,455.0,650.0,Sterling Silver & 18K Gold,Yellow,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1307,Effy Seaside Sterling Silver and 18K Gold Turtle Ring,385.0,550.0,Sterling Silver & 18K Gold,Sterling Silver & 18K Gold,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1430,Effy 925 Sterling Silver and 18K Gold Ring,206.5,295.0,Sterling Silver & 18K Gold,Yellow,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1489,Effy 925 Sterling Silver and 18K Yellow Gold Accented Ring,242.5,485.0,Sterling Silver & 18K Gold,Yellow,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1613,Effy Seaside Sterling Silver & 18K Gold Anchor Bangle,385.0,550.0,Sterling Silver & 18K Gold,Sterling Silver & 18K Gold,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1745,Effy 925 Sterling Silver and 18K Gold Bangle,626.5,895.0,Sterling Silver & 18K Gold,Yellow,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
raw_data.loc[raw_data['Stones'].isnull(), 'Stones'] = 'No Stones'

In [28]:
raw_data.loc[raw_data['Description'] == 'Effy 925 Sterling Silver and 18K Yellow Gold Accented Hoop Earrings']

,Description,Discount_Price,Price,Metal,Metal Color,Stones,Length,Hoop Size,"Height 3/8"" x Length",Diameter,...,"18"" Chain Included. Heart Length","18"" Chain Included. Size of Heart","16"" - 18"" Adjustable Chain. Butterfly Size",Necklace Length,"16""-18"" Adjustable Chain Included. Pendant Legnth","17"" Chain Included. Pendant Length","17"" Attached Chain. Pendant Length","4 Strand 18"" Necklace. Pearl Sizes",Pendant Length,"25"" Gemstone"
373,Effy 925 Sterling Silver and 18K Yellow Gold Accented Hoop Earrings,385.0,550.0,Sterling Silver & 18K Gold,Yellow,No Stones,NaN,NaN,NaN,"1.25""",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
raw_data.loc[raw_data['Stones'].isnull()]

,Description,Discount_Price,Price,Metal,Metal Color,Stones,Length,Hoop Size,"Height 3/8"" x Length",Diameter,...,"18"" Chain Included. Heart Length","18"" Chain Included. Size of Heart","16"" - 18"" Adjustable Chain. Butterfly Size",Necklace Length,"16""-18"" Adjustable Chain Included. Pendant Legnth","17"" Chain Included. Pendant Length","17"" Attached Chain. Pendant Length","4 Strand 18"" Necklace. Pearl Sizes",Pendant Length,"25"" Gemstone"


## Observation on Gemstones column
Only 9 rows have additional information on Gemstones column. 'Gemstones' column can be ignored from the dataframe

In [30]:
raw_data.loc[raw_data['Gemstones'].notnull()][['Description','Discount_Price', 'Price','Metal','Metal Color','Stones','Gemstones']]


,Description,Discount_Price,Price,Metal,Metal Color,Stones,Gemstones
89,"Effy Mosaic Sterling Silver & 18K Gold Multi Gemstone Earrings, 2.75 TCW",455.0,650.0,Sterling Silver & 18K Gold,Yellow,Multi Shape Multi Color 2.75,"Amethyst, Blue Topaz, Citrine, Garnet, Pink Tourmaline, Peridot"
382,"Effy Mosaic 14K Yellow Gold Multi Gemstone Hoop Earrings, 4.05 TCW",945.0,1350.0,14K Gold,Yellow,Oval Multi Color 4.05,"Amethyst, Blue Topaz, Citrine & Peridot"
513,"Effy Watercolors 14K White Gold Multi Gemstone and Diamond Ring, 2.45 TCW",1886.5,2695.0,14K Gold,White,"Multi Shape Multi Color 2.29, Round Diamond 0.16","Amethyst, Sapphire and Tsavorite"
514,"Effy Watercolors 14K White Gold Multi Gemstone and Diamond Ring, 3.52 TCW",2586.5,3695.0,14K Gold,White,"Multi Shape Multi Color 3.38, Round Diamond 0.14","Amethyst, Sapphire and Tsavorite"
825,"Effy Mosaic 14K Yellow Gold Multi Gemstone Ring, 1.90 TCW",556.5,795.0,14K Gold,Yellow,Oval Multi Color 1.9,"Amethyst, Blue Topaz, Citrine & Peridot"
1304,"Effy Mosaic 14K Yellow Gold Multi Gemstone Ring, 1.40 TCW",945.0,1350.0,14K Gold,Yellow,Round Multi Color 1.4,"Amethyst, Blue Topaz, Citrine, Peridot & Garnet."
1448,"Effy Seaside Sterling Silver Multi Gemstone Starfish Ring, 5.66 TCW",497.5,995.0,Sterling Silver & 18K Gold,Yellow,Multi Shape Multi Color 5.66,"Amethyst, Blue Topaz, Citrine, Rhodolite, Peridot"
1891,"Effy Seaside Sterling Silver Multi Gemstone Starfish Pendant, 4.50 TCW",556.5,795.0,Sterling Silver & 18K Gold,Yellow,Multi Shape Multi Color 4.5,"Amethyst, Blue Topaz, Citrine, Rhodolite, Peridot"
2291,"Effy 14K Yellow Gold Multi Gemstone Station Necklace, 28.93 TCW",2695.0,3850.0,14K Gold,Yellow,Round Multi Color 28.93,"Blue Topaz, Citrine, Garnet and Peridot"


## Creating new dataframe

In [31]:
new_raw_data = pd.DataFrame(raw_data[['Description','Discount_Price', 'Price','Metal','Metal Color','Stones']])

In [32]:
new_raw_data.head()

,Description,Discount_Price,Price,Metal,Metal Color,Stones
0,"Effy Brasilica 14K Yellow Gold Emerald and Diamond Earrings, 4.62 TCW",5596.5,7995.0,14K Gold,Yellow,"Multi Shape Emerald 4.01, Round Diamond 0.61"
1,"Effy Pave Classica 14K White Gold Diamond Hoop Earrings, 0.98 TCW",2796.5,3995.0,14K Gold,White,"Round Diamond 0.98,"
2,"Effy Brasilica 14K White Gold Emerald and Diamond Drop Earrings, 4.04 TCW",6996.5,9995.0,14K Gold,Yellow,"Round Diamond 1.38, Pear Emerald 2.66,"
3,"Effy Canare 14K Yellow Gold Yellow Diamond Hoop Earrings, 1.15 TCW",3955.0,5650.0,14K Gold,Yellow,Round Diamond 1.15
4,"Effy Tanzanite Royale 14K Gold Tanzanite and Diamond Earrings, 2.90 TCW",2905.0,4150.0,14K Gold,White,"Multi Shape Tanzanite 2.6, Round Diamond 0.3"


In [33]:
type(new_raw_data)

pandas.core.frame.DataFrame

In [34]:
len(new_raw_data)

2458

## Extract the product weight from the Description

In [35]:
product_description = new_raw_data['Description']

In [36]:
len(product_description)

2458

In [193]:
## Regex 
#m = re.search('.*,\s*(.+?)\s*T?CW\s?$', product)  --> (matches the T specifically in TCW)
#.*  matches any number of characters
#,\s* matches a comma then any number of whitespaces
#(.+?) is the group you originally defined
#\s* matches any number of whitespaces
#T?CW matches a T if it's there, then the CW
#\s?$ matches any number of whitespaces (or none) until the end of the line.

#re.findall("\d+\.\d+", text)

In [176]:
product_weight = []

for product in product_description:
    m = None 
    m = re.search('.*,\s*(.+?)\s*.?CW\s*$', product)
    
    if m:
        prod_weight = float(m.group(1))
    else:
        prod_weight = 0.0 
  
    product_weight.append(prod_weight)

In [177]:
len(product_weight)

2458

In [178]:
product_weight

[4.62,
 0.98,
 4.04,
 1.15,
 2.9,
 0.93,
 2.3,
 1.62,
 0.36,
 0.51,
 0.57,
 1.57,
 0.71,
 0.55,
 3.6,
 3.75,
 0.49,
 6.04,
 6.18,
 2.74,
 2.23,
 2.44,
 1.54,
 2.19,
 1.52,
 0.93,
 0.45,
 0.72,
 0.61,
 3.97,
 0.41,
 1.26,
 0.71,
 0.37,
 1.27,
 0.0,
 2.22,
 1.27,
 0.87,
 3.18,
 0.93,
 0.5,
 0.5,
 4.28,
 3.26,
 1.08,
 2.63,
 5.78,
 0.05,
 1.19,
 1.19,
 1.57,
 0.48,
 0.49,
 1.06,
 2.13,
 9.25,
 8.64,
 2.06,
 3.92,
 1.38,
 2.28,
 0.47,
 0.39,
 2.58,
 0.61,
 0.73,
 2.34,
 0.94,
 0.49,
 2.72,
 4.8,
 0.0,
 0.5,
 0.82,
 0.75,
 22.46,
 2.95,
 0.47,
 1.1,
 7.56,
 0.79,
 6.95,
 1.03,
 0.76,
 1.7,
 0.0,
 2.75,
 5.22,
 0.49,
 0.68,
 0.92,
 1.03,
 1.55,
 0.91,
 0.0,
 1.54,
 1.71,
 0.94,
 0.51,
 1.03,
 0.59,
 1.03,
 0.48,
 0.5,
 0.31,
 1.11,
 5.21,
 1.62,
 0.22,
 0.93,
 0.69,
 0.76,
 2.88,
 0.52,
 1.96,
 11.67,
 0.76,
 0.58,
 2.51,
 0.18,
 1.63,
 4.47,
 4.14,
 1.3,
 0.84,
 0.62,
 4.18,
 1.17,
 1.9,
 0.5,
 1.16,
 0.31,
 0.0,
 0.0,
 0.65,
 0.12,
 5.13,
 0.0,
 0.57,
 1.76,
 3.34,
 0.64,
 0.0,
 0.58,
 0.8

In [179]:
clean_raw_data = new_raw_data.copy()

In [180]:
clean_raw_data.head(1)

,Description,Discount_Price,Price,Metal,Metal Color,Stones
0,"Effy Brasilica 14K Yellow Gold Emerald and Diamond Earrings, 4.62 TCW",5596.5,7995.0,14K Gold,Yellow,"Multi Shape Emerald 4.01, Round Diamond 0.61"


In [181]:
clean_raw_data['Product_Weight'] = product_weight

In [182]:
clean_raw_data.loc[clean_raw_data['Description'] == 'Effy 14K Yellow Gold Cultured Fresh Water Pearl Earrings']

,Description,Discount_Price,Price,Metal,Metal Color,Stones,Product_Weight
88,Effy 14K Yellow Gold Cultured Fresh Water Pearl Earrings,315.0,450.0,14K Gold,Yellow,Round Pearl,0.0
97,Effy 14K Yellow Gold Cultured Fresh Water Pearl Earrings,416.5,595.0,14K Gold,Yellow,Round Pearl,0.0


In [183]:
df = clean_raw_data.copy()

In [187]:
df.to_csv("/Users/rupalgandhre/SpringBoard/DataScience_Capstone2/data/raw/effy_clean_raw_data.csv", index=False, header=True)

In [185]:
df

,Description,Discount_Price,Price,Metal,Metal Color,Stones,Product_Weight
0,"Effy Brasilica 14K Yellow Gold Emerald and Diamond Earrings, 4.62 TCW",5596.5,7995.0,14K Gold,Yellow,"Multi Shape Emerald 4.01, Round Diamond 0.61",4.62
1,"Effy Pave Classica 14K White Gold Diamond Hoop Earrings, 0.98 TCW",2796.5,3995.0,14K Gold,White,"Round Diamond 0.98,",0.98
2,"Effy Brasilica 14K White Gold Emerald and Diamond Drop Earrings, 4.04 TCW",6996.5,9995.0,14K Gold,Yellow,"Round Diamond 1.38, Pear Emerald 2.66,",4.04
3,"Effy Canare 14K Yellow Gold Yellow Diamond Hoop Earrings, 1.15 TCW",3955.0,5650.0,14K Gold,Yellow,Round Diamond 1.15,1.15
4,"Effy Tanzanite Royale 14K Gold Tanzanite and Diamond Earrings, 2.90 TCW",2905.0,4150.0,14K Gold,White,"Multi Shape Tanzanite 2.6, Round Diamond 0.3",2.90
...,...,...,...,...,...,...,...
2466,"Effy Mosaic 14K Yellow Gold Multi Gemstone and Diamond Pendant, 1.18 TCW",800.0,1600.0,14K Gold,Yellow,"Square Multi Color 1.1, Round Diamond 0.08",1.18
2467,"Effy Bordeaux 14K Yellow Gold Garnet and Diamond Pendant, 8.14 TCW",2205.0,3150.0,14K Gold,Yellow,"Emerald Cut Garnet 7.85, Multi Shape Diamond 0.29",8.14
2468,"Effy Watercolors 14K Yellow Gold Multi Sapphire & Diamond Pendant, 1.85 TCW",2306.5,3295.0,14K Gold,Yellow,"Oval Multi Sapphire 1.59, Round Diamond 0.26",1.85
2469,"Effy Trio 14K Tri Color Gold Diamond Necklace, 0.40 TCW",1606.5,2295.0,14K Gold,Three Tone,Multi Shape Diamond 0.4,0.40
